<a href="https://colab.research.google.com/github/Mdawood1122/MSC-Dissertation/blob/main/dissertation_rice_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import time
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import plot_tree
import seaborn as sb

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/rice_new.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/rice_new.csv'

In [ ]:
X = df.drop(columns = ['Class'])              #drop 'Class' variable and move all other variable in X
y = df['Class'].values                        # assign target class to y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)   #train test split 30% training, 70% testing

In [ ]:
df.info()

In [ ]:
 df['Class'].value_counts()

In [ ]:
sb.countplot(x='Class',data=df)            # plot labels

In [ ]:
df.isnull().sum()                            # check for missing values

**Evaluation of model on Default hyperparamters value**

In [ ]:
RFC = RandomForestClassifier()                        #Create Random Forest Classifier
RFC_rice=RFC.fit(X_train, y_train)                    #train model on training set
RFC_rice.score(X_test,y_test)                         #test model on test set

In [ ]:
svm_classi = svm.SVC()                                #create SVC model
SVC_rice=svm_classi.fit(X_train, y_train)             #training of model
SVC_rice.score(X_test,y_test)                         # testing of model

In [ ]:
knn = KNeighborsClassifier()                          #create KNN model
KNN_rice=knn.fit(X_train,y_train)                     #training of knn
KNN_rice.score(X_test,y_test)                         #testing

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have trained the KNN classifier and obtained 'model'

# 1. Make predictions on the test data
y_pred = RFC_rice.predict(X_test)

# 2. Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)

# 3. Visualize the confusion matrix using a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
# 4. Show the classification report
print(classification_report(y_test, y_pred))

In [ ]:
pip install scikit-optimize

**Bayesian Search for SVC**

In [ ]:
from skopt import BayesSearchCV
param_space = {
    'C': (0.1, 1.0, 'log-uniform'),          # Smaller range for C
    'kernel': ['linear', 'rbf'],
    'gamma': (1e-6, 1e-2, 'log-uniform')     # Smaller range for gamma
}
start_time = time.time()                     #start timer
opt = BayesSearchCV(SVC_rice, param_space, n_iter=10, cv=5, n_jobs=-1)     #Bayesian Optimization algorithm
opt.fit(X_train, y_train)                    #train optimiztion algorithm
end_time = time.time()
execution_time = end_time - start_time       #find execution time of algorithm
best_params = opt.best_params_               #Best hyperparameters value
best_model = opt.best_estimator_
test_accuracy = best_model.score(X_test, y_test)    #evaluation of model on test dataset
print('Test accuracy:', test_accuracy)
print('Time taken (in seconds):', execution_time)

**Bayesian search for KNN**

In [ ]:
from skopt import BayesSearchCV

param_space = {
    'n_neighbors': (1, 35),              # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weight function
    'p': (1, 2)                          # Power parameter (1 for Manhattan, 2 for Euclidean)
}
start_time = time.time()
opt = BayesSearchCV(KNN_rice, param_space, n_iter=50, cv=5)   # Perform Bayesian Optimization
opt.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time
best_params = opt.best_params_           # Best hyperparameters and best model
best_model = opt.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print('Test accuracy:', test_accuracy)     #accuracy of the model on the test dataset
print('Time taken (in seconds):', execution_time)

**Random Search for SVC**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.1, 1.0, 10.0],                          #candidate value for C
    'kernel': ['linear', 'rbf'],                    #list of kernal
    'gamma': [0.001, 0.01, 0.1, 'scale', 'auto']    #vlaues of gamma
}
start_time = time.time()                            #start timer
grid_search = GridSearchCV(SVC_rice, param_grid, cv=5) # creation Grid Search
grid_search.fit(X_train, y_train)                       #training of model
end_time = time.time()
execution_time = end_time - start_time              #time grid search take to execute
best_params = grid_search.best_params_              #best hyperparameter values
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test) # Evaluate the best model on the test data
print('Test accuracy:', test_accuracy)
print('Time taken (in seconds):', execution_time)

**Random Search for KNN**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
param_dist = {
    'n_neighbors': randint(1,50 ),           #Number of neighbors
    'weights': ['uniform', 'distance'],      # Weight function
    'p': [1, 20]                             #Power parameter (1 for Manhattan, 2 for Euclidean)
}
start_time = time.time()
random_search = RandomizedSearchCV(KNN_rice, param_distributions=param_dist, n_iter=150, cv=20) # creation of Random search algorithm
random_search.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time     #calculation of execution time
best_params = random_search.best_params_   #best hyperparameters value
best_model = random_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test) # Evaluate the best model on the test data
print('Test accuracy:', test_accuracy)
print('Time taken (in seconds):', execution_time)

**Grid Search for SVC**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.001, 0.01, 0.1, 'scale', 'auto']
}
start_time = time.time()
grid_search = GridSearchCV(SVC_rice, param_grid, cv=5)          #creation of Grid Search algorithm
grid_search.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time                          #calculation of execution time
best_params = grid_search.best_params_                          # Best hyperparameters values
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)                #evaluation of the optimized algorithm on test dataset
print('Test accuracy:', test_accuracy)
print('Time taken (in seconds):', execution_time)

**Grid Search for KNN**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_neighbors': range(1, 50),         # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weight function
    'p': [1, 3]                          # Power parameter (1 for Manhattan, 2 for Euclidean)
}
start_time = time.time()
grid_search = GridSearchCV(KNN_rice, param_grid, cv=5)   #Grid Search algorithm
grid_search.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time                   #execution time of grid search
best_params = grid_search.best_params_                   # Best hyperparameters and best model
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)         #model evalution on the test dataset
print('Test accuracy:', test_accuracy)
print('Time taken (in seconds):', execution_time)